# Rendering Thermal Videos of Gas
Using this notebook will allow you to create 128x128 pixel videos of gas, such as [this example here](https://github.com/max-wild/Thermal-Gas-Videos/tree/main/examples):

<br>

<center><img src="https://github.com/max-wild/Thermal-Gas-Videos/raw/main/examples/example.gif"></center>

<br>

First, you need to **edit** and **run** the contents of the first cell to

*   Let this notebook access google drive files
*   Set up the parameters for video generation 

    &emsp;&emsp; (e.g. slideshow of .png images versus one .mp4 file)




In [ ]:
#@title Parameters for Video Generation


# How many frames per second are used in the video
fps = "30"  #@param [24, 25, 30, 50, 60, 120, 240] {allow-input: true}

# How many seconds are rendered
seconds = 5  #@param {type: "integer"}

# How many videos of different gas scenarios are rendered
video_count = 1  #@param {type: "integer"}

# Generate a collection of gas images instead of a cohesive gas video
render_image_slideshow = False  #@param {type: "boolean"}

output_dir = '/content/drive/MyDrive/gas_videos/'  #@param {type: "string"}


# Getting access to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Setting Up the Environment

In [ ]:
# Asserting the GPU is enabled
import tensorflow as tf
assert(tf.test.gpu_device_name() == '/device:GPU:0')

fps = round(fps)
seconds = round(seconds)
video_count = round(video_count)

In [ ]:
# Fix for a known bug when using blender with colab
import os

os.environ["LD_PRELOAD"] = ""

!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4

os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"
!apt install libboost-all-dev
!apt install libgl1-mesa-dev 
!apt install libglu1-mesa libsm-dev

## Getting Necessary Files

In [ ]:
%%shell
# -------- Installing blender -------- #

BLEND_DIR="/blender-3.2.2-linux-x64/"

# Download blender zip 
if [ ! -f "blender-3.2.2-linux-x64.tar.xz" ]; then
  wget -nc https://download.blender.org/release/Blender3.2/blender-3.2.2-linux-x64.tar.xz
fi

# If the blend tar hasn't been unzipped, then unzip it
[ ! -d "$BLEND_DIR" ] && tar -xf blender-3.2.2-linux-x64.tar.xz

--2022-08-23 15:29:06--  https://download.blender.org/release/Blender3.2/blender-3.2.2-linux-x64.tar.xz
Resolving download.blender.org (download.blender.org)... 82.94.213.221
Connecting to download.blender.org (download.blender.org)|82.94.213.221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191760080 (183M) [application/octet-stream]
Saving to: ‘blender-3.2.2-linux-x64.tar.xz’

blender-3.2.2-linux 100%[===================>] 182.88M  6.65MB/s    in 21s     

2022-08-23 15:29:26 (8.92 MB/s) - ‘blender-3.2.2-linux-x64.tar.xz’ saved [191760080/191760080]



In [ ]:
# Get the rendering files

!mkdir setup
%cd setup 

!wget https://github.com/max-wild/Thermal-Gas-Videos/raw/main/gas_scene.blend
!wget https://github.com/max-wild/Thermal-Gas-Videos/raw/main/render_video.py

%cd /content/

/content/setup
--2022-08-23 16:49:10--  https://github.com/max-wild/Thermal-Gas-Videos/raw/main/gas_scene.blend
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/max-wild/Thermal-Gas-Videos/main/gas_scene.blend [following]
--2022-08-23 16:49:10--  https://raw.githubusercontent.com/max-wild/Thermal-Gas-Videos/main/gas_scene.blend
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1100492 (1.0M) [application/octet-stream]
Saving to: ‘gas_scene.blend’

gas_scene.blend     100%[===================>]   1.05M  --.-KB/s    in 0.02s   

2022-08-23 16:49:10 (59.4 MB/s) - ‘gas_scene.blend’ saved [1100492/11004

## Shell Command to Render Thermal Gas Videos

In [ ]:
blend_dir = '/content/blender-3.2.2-linux-x64/blender'

!{blend_dir} setup/gas_scene.blend --background --python setup/render_video.py -- -f {fps} -s {seconds} -v {video_count} -r {render_image_slideshow} -o {output_dir}